In [3]:
import string
from collections import Counter, defaultdict
import random

import requests

In [4]:
DATA_URL = 'https://wolnelektury.pl/media/book/txt/nasza-szkapa.txt'

In [5]:
text = requests.get(DATA_URL).text
text

'Maria Konopnicka\r\n\r\nNasza szkapa\r\n\r\nISBN 978-83-288-2363-1\r\n\r\n\r\nZaczęło się to od starego łóżka, cośmy na nim we trzech sypiali.\r\n\r\nTego dnia ojciec zły czegoś z rzeki wrócił i, siadłszy na ławie, ręką głowę potarł. Pytała się matka raz i drugi, co mu, ale dopiero za trzecim razem odpowiedział, że się ta robota koło żwiru skończyła i że szkapa tylko piasek wozić będzie. Zaraz mnie Felek szturchnął w bok, a matka jęknęła z cicha.\r\n\r\nMiał ojciec nad wieczorem po doktora iść, ale mu jakoś niesporo było. Chodził, medytował, po kątach pozierał, aż stanął przed matką i rzekł:\r\n\r\n— Co chłopakom po łóżku, Anulka? Sypiam ja na ziemi, toż i oni mogą.\r\n\r\nSpojrzeliśmy po sobie. Dwie złote iskry zabłysły w siwych oczach Felka. Prawda! Co nam po łóżku? Piotrusia tylko pilnować trzeba, żeby z niego nie spadł.\r\n\r\n— Dalej! jazda! — krzyknął Felek, i, zanim odpowiedzieć zdążyła, jużeśmy we trzech siennik na ziemię ściągnęli, a Felek kozły wywracać na nim zaczął.\r\n\r\

In [6]:
prefix = 'Maria Konopnicka Nasza szkapa ISBN 978-83-288-2363-1'

text = text.replace('\r\n', ' ')

text = text.replace('  ', ' ')
text = text[len(prefix):]
text = text.split('-----')[0]

# in python 3.9 + 
# text.removeprefix(prefix)

In [7]:
import re


def sentence_split_using_re(text):
    sentence_terminators = '.!?'
    sentence_terminators = re.compile('[.!?]')
    return sentence_terminators.split(text)


def sentence_split(text):
    sentence_terminators = '.!?'
    current_sentence = ''
    sentences = []
    for char in text:
        if char in sentence_terminators:
            if len(current_sentence) > 0:
                sentences.append(current_sentence)
                current_sentence = ''
        else:
            current_sentence += char
    if len(current_sentence) > 0:
        sentences.append(current_sentence)
    return sentences

In [8]:
%%timeit
sentence_split(text)

2.62 ms ± 7.68 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
sentence_split_using_re(text)

287 µs ± 401 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [10]:
def tokenize(sentence):
    for punct in string.punctuation:
        sentence = sentence.replace(punct, ' ')
    tokenized = [t for t in sentence.lower().split() if t.isalpha() and len(t)]
    return tokenized

In [11]:
tokenized = [tokenize(sentence) for sentence in sentence_split(text)]
tokenized

[['zaczęło',
  'się',
  'to',
  'od',
  'starego',
  'łóżka',
  'cośmy',
  'na',
  'nim',
  'we',
  'trzech',
  'sypiali'],
 ['tego',
  'dnia',
  'ojciec',
  'zły',
  'czegoś',
  'z',
  'rzeki',
  'wrócił',
  'i',
  'siadłszy',
  'na',
  'ławie',
  'ręką',
  'głowę',
  'potarł'],
 ['pytała',
  'się',
  'matka',
  'raz',
  'i',
  'drugi',
  'co',
  'mu',
  'ale',
  'dopiero',
  'za',
  'trzecim',
  'razem',
  'odpowiedział',
  'że',
  'się',
  'ta',
  'robota',
  'koło',
  'żwiru',
  'skończyła',
  'i',
  'że',
  'szkapa',
  'tylko',
  'piasek',
  'wozić',
  'będzie'],
 ['zaraz',
  'mnie',
  'felek',
  'szturchnął',
  'w',
  'bok',
  'a',
  'matka',
  'jęknęła',
  'z',
  'cicha'],
 ['miał',
  'ojciec',
  'nad',
  'wieczorem',
  'po',
  'doktora',
  'iść',
  'ale',
  'mu',
  'jakoś',
  'niesporo',
  'było'],
 ['chodził',
  'medytował',
  'po',
  'kątach',
  'pozierał',
  'aż',
  'stanął',
  'przed',
  'matką',
  'i',
  'rzekł',
  'co',
  'chłopakom',
  'po',
  'łóżku',
  'anulka'],
 ['sy

In [12]:
def get_ngrams(tokens, n):
    t = ['<START>'] * (n - 1) + tokens
    return [(tuple(t[i:i+n-1]), t[i+n]) for i in range(len(t)-n)]

In [13]:
n_grams = [get_ngrams(sentence, 3) for sentence in tokenized]
# Counter(n_grams).most_common()
n_grams

[[(('<START>', '<START>'), 'się'),
  (('<START>', 'zaczęło'), 'to'),
  (('zaczęło', 'się'), 'od'),
  (('się', 'to'), 'starego'),
  (('to', 'od'), 'łóżka'),
  (('od', 'starego'), 'cośmy'),
  (('starego', 'łóżka'), 'na'),
  (('łóżka', 'cośmy'), 'nim'),
  (('cośmy', 'na'), 'we'),
  (('na', 'nim'), 'trzech'),
  (('nim', 'we'), 'sypiali')],
 [(('<START>', '<START>'), 'dnia'),
  (('<START>', 'tego'), 'ojciec'),
  (('tego', 'dnia'), 'zły'),
  (('dnia', 'ojciec'), 'czegoś'),
  (('ojciec', 'zły'), 'z'),
  (('zły', 'czegoś'), 'rzeki'),
  (('czegoś', 'z'), 'wrócił'),
  (('z', 'rzeki'), 'i'),
  (('rzeki', 'wrócił'), 'siadłszy'),
  (('wrócił', 'i'), 'na'),
  (('i', 'siadłszy'), 'ławie'),
  (('siadłszy', 'na'), 'ręką'),
  (('na', 'ławie'), 'głowę'),
  (('ławie', 'ręką'), 'potarł')],
 [(('<START>', '<START>'), 'się'),
  (('<START>', 'pytała'), 'matka'),
  (('pytała', 'się'), 'raz'),
  (('się', 'matka'), 'i'),
  (('matka', 'raz'), 'drugi'),
  (('raz', 'i'), 'co'),
  (('i', 'drugi'), 'mu'),
  (('drugi'

In [18]:
class NgramModel(object):

    def __init__(self, n):
        self.n = n
        self.context = defaultdict(list)
        self.ngram_counter = Counter()

    def update(self, sentence: str) -> None:
        ngrams = get_ngrams(tokenize(sentence), self.n)
        for ngram in ngrams:
            self.ngram_counter[ngram] += 1
            self.context[ngram[0]].append(ngram[1])
                
    def prob(self, context, token):
        """
        Calculates probability of a candidate token to be generated given a context
        :return: conditional probability
        """
        count_of_token = self.ngram_counter[(context, token)]
        count_of_context = len(self.context[context])
        if count_of_context > 0: 
            return count_of_token / count_of_context
        return 0.0
    
    def random_token(self, context):
        """
        Given a context we "semi-randomly" select the next word to append in a sequence
        :param context:
        :return:
        """
        r = random.random()
        map_to_probs = {}
        token_of_interest = self.context[context]
        for token in token_of_interest:
            map_to_probs[token] = self.prob(context, token)

        summ = 0
        for token in sorted(map_to_probs):
            summ += map_to_probs[token]
            if summ > r:
                return token

    def generate_text(self, token_count: int):
        """
        :param token_count: number of words to be produced
        :return: generated text
        """
        n = self.n
        context_queue = (n - 1) * ['<START>']
        result = []
        while len(result) < token_count:
            predicted_token = self.random_token(tuple(context_queue))
            if predicted_token:
                result.append(predicted_token)
            else:
                predicted_token = '<START>'
            context_queue.pop(0)
            context_queue.append(predicted_token)
        return ' '.join(result)

In [19]:
%%time
model = NgramModel(2)
for sentence in sentence_split(text):
    model.update(sentence) 

CPU times: user 34 ms, sys: 4.47 ms, total: 38.4 ms
Wall time: 36.5 ms


In [20]:
model.ngram_counter.most_common()

[((('<START>',), 'się'), 55),
 ((('<START>',), 'to'), 38),
 ((('<START>',), 'felek'), 24),
 ((('i',), 'się'), 18),
 ((('<START>',), 'ojciec'), 16),
 ((('<START>',), 'na'), 15),
 ((('i',), 'z'), 13),
 ((('<START>',), 'tylko'), 11),
 ((('<START>',), 'i'), 11),
 ((('<START>',), 'nie'), 11),
 ((('się',), 'na'), 11),
 ((('<START>',), 'w'), 10),
 ((('w',), 'i'), 10),
 ((('<START>',), 'też'), 10),
 ((('<START>',), 'ja'), 9),
 ((('<START>',), 'z'), 9),
 ((('<START>',), 'mnie'), 8),
 ((('to',), 'na'), 8),
 ((('<START>',), 'matka'), 8),
 ((('<START>',), 'jak'), 8),
 ((('felek',), 'się'), 8),
 ((('<START>',), 'dnia'), 7),
 ((('i',), 'na'), 7),
 ((('z',), 'na'), 7),
 ((('nie',), 'a'), 7),
 ((('na',), 'i'), 7),
 ((('a',), 'do'), 7),
 ((('a',), 'to'), 7),
 ((('a',), 'się'), 7),
 ((('<START>',), 'ty'), 7),
 ((('<START>',), 'toć'), 7),
 ((('się',), 'i'), 7),
 ((('<START>',), 'jej'), 7),
 ((('<START>',), 'mi'), 7),
 ((('w',), 'na'), 7),
 ((('<START>',), 'że'), 7),
 ((('<START>',), 'co'), 7),
 ((('do',)

In [31]:
model.generate_text(10)

na
na
ulicy
ulicy
szanowałem
szanowałem
moździerz
moździerz
żelazko
żelazko
były
były
klejnoty
klejnoty
None
dopiero
dopiero
do
do
rana
rana


'na ulicy szanowałem moździerz żelazko były klejnoty dopiero do rana'